<a href="https://colab.research.google.com/github/Kulkarni-mohit/Fine-tuning-sentiment-analysis/blob/main/Fine_Tuned_Sentiment_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Downloading and importing libraries

In [1]:
! pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import tensorflow as tf
import transformers
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification

## Loading and Tokenizing Dataset

In [3]:
from datasets import load_dataset

dataset = load_dataset("imdb")
dataset["train"][100]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating unsupervised split: 0 examples [00:00, ? examples/s]

{'text': "Terrible movie. Nuff Said.<br /><br />These Lines are Just Filler. The movie was bad. Why I have to expand on that I don't know. This is already a waste of my time. I just wanted to warn others. Avoid this movie. The acting sucks and the writing is just moronic. Bad in every way. The only nice thing about the movie are Deniz Akkaya's breasts. Even that was ruined though by a terrible and unneeded rape scene. The movie is a poorly contrived and totally unbelievable piece of garbage.<br /><br />OK now I am just going to rag on IMDb for this stupid rule of 10 lines of text minimum. First I waste my time watching this offal. Then feeling compelled to warn others I create an account with IMDb only to discover that I have to write a friggen essay on the film just to express how bad I think it is. Totally unnecessary.",
 'label': 0}

In [4]:
MODEL_NAME = 'distilbert-base-uncased-finetuned-sst-2-english'
BATCH_SIZE = 16
N_EPOCHS = 1

In [5]:
X_train = dataset["train"]
X_test = dataset["test"]

In [6]:
#define a tokenizer object
tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)
#tokenize the text
train_encodings = tokenizer(list(X_train["text"]),
                            truncation=True,
                            padding=True)
test_encodings = tokenizer(list(X_test["text"]),
                           truncation=True,
                           padding=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

In [7]:
train_encodings

In [8]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings),
                                    list(X_train["label"])))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings),
                                    list(X_test["label"])))

In [ ]:
train_dataset

## Fine Tuning Model



In [9]:
model = TFDistilBertForSequenceClassification.from_pretrained(MODEL_NAME)
#chose the optimizer
optimizerr = tf.keras.optimizers.Adam(learning_rate=5e-5)
#define the loss function
losss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
#build the model
model.compile(optimizer=optimizerr,
              loss=losss,
              metrics=['accuracy'])
# train the model
model.fit(train_dataset.shuffle(len(X_train)).batch(BATCH_SIZE),
          epochs=N_EPOCHS,
          batch_size=BATCH_SIZE)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


1563/1563 [==============================] - 1425s 894ms/step - loss: 0.2416 - accuracy: 0.9042


## Evaluating the Model

In [10]:
model.evaluate(test_dataset.shuffle(len(X_test)).batch(BATCH_SIZE),
               return_dict=True,
               batch_size=BATCH_SIZE)

1563/1563 [==============================] - 494s 314ms/step - loss: 0.1914 - accuracy: 0.9261


{'loss': 0.19138438999652863, 'accuracy': 0.9260799884796143}

In [21]:
prompt = ["The movie was so good.", "The movie was not so good."]

In [22]:
prompt_encoding = tokenizer(prompt,
                            truncation=True,
                            padding=True)

In [23]:
prompt_trans = tf.data.Dataset.from_tensor_slices((dict(prompt_encoding)))

In [24]:
preds = model.predict(prompt_trans.batch(1)).logits

2/2 [==============================] - 0s 16ms/step


In [25]:
tf.nn.softmax(preds, axis=1).numpy()

array([[0.00166763, 0.99833244],
       [0.99600416, 0.00399584]], dtype=float32)